In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import matplotlib.pyplot as plt

data = pd.read_excel("Online Retail.xlsx")
data.info()

KeyboardInterrupt: 

In [6]:
# 1. splitting the dataset according to the given date 
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])

# Define the start and end date for the subset
start_date = pd.to_datetime("2011-09-01")
end_date = pd.to_datetime("2011-11-30")

# Create a subset of data for the specified date range
subset_data = data[(data["InvoiceDate"] >= start_date) & (data["InvoiceDate"] <= end_date)]
print(subset_data.size)

1537800


In [27]:
# 2. Transform the above data into new dataset
grouped_data = subset_data.groupby('InvoiceNo')
invoice_data = grouped_data['StockCode'].apply(list).reset_index()
print("New data\n", invoice_data)

invoice_data.to_csv('12012008.csv', index=False)

New data
      InvoiceNo                                          StockCode
0       565080                                     [20677, 22128]
1       565081                                            [21067]
2       565082                             [22423, 15060B, 23245]
3       565083  [22609, 22741, 23146, 72351A, 72351B, 22652, 2...
4       565084         [23309, 22970, 22988, 22902, 22659, 22616]
...        ...                                                ...
8257   C579438                                            [21533]
8258   C579496                                            [22666]
8259   C579502                                            [22946]
8260   C579519                                            [23139]
8261   C579521                              [22307, 21843, 22840]

[8262 rows x 2 columns]


In [40]:
#3. Prepare the data and generate association rules 

# Explode the lists in the "StockCode" column to create one row per item in an invoice
invoice_data = invoice_data.explode('StockCode')

# Pivot the data to get a binary matrix where columns represent unique items (StockCode) and rows represent transactions (invoices)
basket = pd.crosstab(invoice_data['InvoiceNo'], invoice_data['StockCode']).applymap(lambda x: 1 if x > 0 else 0)

# Reset the index and drop any non-numeric columns
basket.reset_index(inplace=True)
# This is giving a memory limit exceed error

C:\Users\hp\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


MemoryError: Unable to allocate 34.8 GiB for an array with shape (282376, 2, 8262) and data type int64

In [61]:
all_txns = []
remove_substrings = lambda s: [item.replace('"[', '').replace(']"', '').replace('[', '').replace(']', '').replace(' ', '').replace('"', '').replace("'", '') for item in s.split(", ")]
with open('12012008.csv') as f:
    #read each line
    content = f.readlines()
    #Remove white space from the beginning and end of the line
    txns = [x.strip() for x in content]
    # Iterate through each line and create a list of transactions
    for each_txn in txns:
        #Each transaction will contain a list of item in the transaction
        split_rows = each_txn.split(',')
        split_rows = list(map(remove_substrings, split_rows))
       
        joined_rows = []
        for split in split_rows:
            joined_rows = joined_rows + split
        joined_rows = ','.join(map(str, joined_rows))
#         print('joined_rows: ', joined_rows)
#         print('split_row: ', split_rows)
        all_txns.append( joined_rows.split(','))
       
all_txns[:5]

[['InvoiceNo', 'StockCode'],
 ['565080', '20677', '22128'],
 ['565081', '21067'],
 ['565082', '22423', '15060B', '23245'],
 ['565083',
  '22609',
  '22741',
  '23146',
  '72351A',
  '72351B',
  '22652',
  '22489',
  '22653',
  '23263',
  '21415',
  '22212']]

In [62]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

encoder = TransactionEncoder()

one_hot_encoding = encoder.fit_transform(all_txns)
one_hot_txns_df = pd.DataFrame(one_hot_encoding, columns=encoder.columns_)

one_hot_txns_df.head(10)


,10080,10120,10124A,10124G,10125,10133,10135,11001,15030,15034,...,DCGSSGIRL,DOT,InvoiceNo,M,PADS,POST,S,StockCode,gift_0001_20,gift_0001_30
0,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [65]:
# Generate frequent itemsets
frequent_itemsets = apriori(one_hot_txns_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Filter rules based on minimum support and lift
filtered_rules = rules[(rules['support'] >= 0.01) & (rules['lift'] > 1.0)]

print(filtered_rules.sample(5))

         antecedents            consequents  antecedent support  \
0            (20712)                (21931)            0.028682   
1            (21931)                (20712)            0.038485   
2            (20712)                (22385)            0.028682   
3            (22385)                (20712)            0.026141   
4            (20712)                (22386)            0.028682   
...              ...                    ...                 ...   
2945  (23265, 23266)         (23263, 23264)            0.021784   
2946         (23263)  (23265, 23264, 23266)            0.038969   
2947         (23264)  (23263, 23265, 23266)            0.028803   
2948         (23265)  (23263, 23264, 23266)            0.028924   
2949         (23266)  (23263, 23265, 23264)            0.030618   

      consequent support   support  confidence       lift  leverage  \
0               0.038485  0.011860    0.413502  10.744553  0.010756   
1               0.028682  0.011860    0.308176  10.74

In [66]:
# 4. Print top 10 rules in descending order
filtered_rules.sort_values( 'confidence',ascending = False)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2220,"(21935, 22961)",(DOT),0.012102,0.016096,0.011860,0.980000,60.885263,0.011665,49.195208,0.995625
2214,"(DOT, 21934)",(21935),0.011739,0.020937,0.011255,0.958763,45.793397,0.011009,23.742285,0.989782
2146,"(22577, 21810)",(22578),0.011134,0.051555,0.010529,0.945652,18.342544,0.009955,17.451386,0.956127
2640,"(23170, 23172)",(23171),0.012102,0.018516,0.011376,0.940000,50.766144,0.011152,16.358062,0.992311
2507,"(DOT, 22666)",(22961),0.010771,0.040905,0.010045,0.932584,22.798650,0.009604,14.226572,0.966548
2459,"(22577, 22579)",(22578),0.028077,0.051555,0.026020,0.926724,17.975403,0.024572,12.943483,0.971649
2841,"(20723, 22356, 20719)",(20724),0.010892,0.042963,0.010045,0.922222,21.465696,0.009577,12.304767,0.963913
2518,"(23171, 22698)",(22697),0.011618,0.032192,0.010650,0.916667,28.475251,0.010276,11.613700,0.976224
2219,"(22961, DOT)",(21935),0.012949,0.020937,0.011860,0.915888,43.745557,0.011589,11.639975,0.989960
2477,"(22577, 23265)",(22578),0.011497,0.051555,0.010529,0.915789,17.763306,0.009936,11.262783,0.954680


In [74]:
#5.  Repeat all above 4 questions for coutnry france only 
# a. splitting the dataset according to the given date
new_data = data[data['Country'] == 'France']
new_data["InvoiceDate"] = pd.to_datetime(new_data["InvoiceDate"])

# Define the start and end date for the subset
start_date = pd.to_datetime("2010-12-01")
end_date = pd.to_datetime("2011-12-09")

# Create a subset of data for the specified date range
subset_data = new_data[(new_data["InvoiceDate"] >= start_date) & (new_data["InvoiceDate"] <= end_date)]
print(subset_data.size)

68336


C:\Users\hp\AppData\Local\Temp\ipykernel_18760\2797019464.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["InvoiceDate"] = pd.to_datetime(new_data["InvoiceDate"])


In [75]:
# b. Transform the above data into new dataset
grouped_data = subset_data.groupby('InvoiceNo')
invoice_data = grouped_data['StockCode'].apply(list).reset_index()
print("New data\n", invoice_data)

invoice_data.to_csv('12012008_France.csv', index=False)

New data
     InvoiceNo                                          StockCode
0      536370  [22728, 22727, 22726, 21724, 21883, 10002, 217...
1      536852   [22549, 22544, 22539, 22661, 21791, 21786, POST]
2      536974  [15056BL, 15056P, 20679, 21915, 22659, 22352, ...
3      537065  [22837, 22846, 22892, 22968, 84678, 20665, 217...
4      537463  [22961, 21224, 22326, 21124, 21121, 21137, 220...
..        ...                                                ...
455   C579532                                      [POST, 22890]
456   C579562                                     [23084, 21731]
457   C580161                                             [POST]
458   C580263  [M, M, 70007, 85175, 84821, 84819, 84817, 7258...
459   C581316                              [23020, 21531, 23174]

[460 rows x 2 columns]


In [76]:
#c
all_txns = []
remove_substrings = lambda s: [item.replace('"[', '').replace(']"', '').replace('[', '').replace(']', '').replace(' ', '').replace('"', '').replace("'", '') for item in s.split(", ")]
with open('12012008_France.csv') as f:
    #read each line
    content = f.readlines()
    #Remove white space from the beginning and end of the line
    txns = [x.strip() for x in content]
    # Iterate through each line and create a list of transactions
    for each_txn in txns:
        #Each transaction will contain a list of item in the transaction
        split_rows = each_txn.split(',')
        split_rows = list(map(remove_substrings, split_rows))
       
        joined_rows = []
        for split in split_rows:
            joined_rows = joined_rows + split
        joined_rows = ','.join(map(str, joined_rows))
#         print('joined_rows: ', joined_rows)
#         print('split_row: ', split_rows)
        all_txns.append( joined_rows.split(','))
       
all_txns[:5]

[['InvoiceNo', 'StockCode'],
 ['536370',
  '22728',
  '22727',
  '22726',
  '21724',
  '21883',
  '10002',
  '21791',
  '21035',
  '22326',
  '22629',
  '22659',
  '22631',
  '22661',
  '21731',
  '22900',
  '21913',
  '22540',
  '22544',
  '22492',
  'POST'],
 ['536852', '22549', '22544', '22539', '22661', '21791', '21786', 'POST'],
 ['536974',
  '15056BL',
  '15056P',
  '20679',
  '21915',
  '22659',
  '22352',
  '22892',
  '22027',
  '20749',
  '20750',
  '21700',
  '22748',
  '22080',
  '20725',
  '20726',
  'POST'],
 ['537065',
  '22837',
  '22846',
  '22892',
  '22968',
  '84678',
  '20665',
  '21786',
  '21787',
  '20750',
  '20749',
  '21774',
  '21121',
  '21439',
  '47566',
  '22727',
  '22726',
  '22728',
  '22729',
  '22243',
  '22467',
  '22318',
  '21195',
  '20725',
  '20726',
  '20728',
  '21559',
  '22326',
  '22352',
  '21555',
  '21533',
  '21539',
  '22423',
  '84997A',
  '84997B',
  '84997C',
  '84279P',
  '22634',
  '22635',
  '22636',
  '22180',
  '21980',
  '219

In [77]:
encoder = TransactionEncoder()

one_hot_encoding = encoder.fit_transform(all_txns)
one_hot_txns_df = pd.DataFrame(one_hot_encoding, columns=encoder.columns_)

one_hot_txns_df.head(10)

,10002,10120,10125,10135,11001,15036,15039,15044C,15056BL,15056N,...,C579192,C579532,C579562,C580161,C580263,C581316,InvoiceNo,M,POST,StockCode
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [78]:
# Generate frequent itemsets
frequent_itemsets = apriori(one_hot_txns_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Filter rules based on minimum support and lift
filtered_rules = rules[(rules['support'] >= 0.01) & (rules['lift'] > 1.0)]

print(filtered_rules.sample(5))

                               antecedents                   consequents  \
40000                 (20726, POST, 22629)                       (22556)   
150808  (21086, 22437, 21988, POST, 21989)  (21080, 21094, 21987, 22435)   
68636                              (22554)          (22551, POST, 22661)   
46265                (21094, 21086, 21124)                       (21121)   
35897                       (22629, 22382)                (22328, 20725)   

        antecedent support  consequent support   support  confidence  \
40000             0.034707            0.145336  0.013015    0.375000   
150808            0.015184            0.013015  0.010846    0.714286   
68636             0.145336            0.028200  0.026030    0.179104   
46265             0.010846            0.065076  0.010846    1.000000   
35897             0.047722            0.021692  0.010846    0.227273   

             lift  leverage  conviction  zhangs_metric  
40000    2.580224  0.007971    1.367462       0.63445

In [79]:
# d. Print top 10 rules in descending order
filtered_rules.sort_values( 'confidence',ascending = False)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
115267,"(22629, 22728, 22727, 22554)",(POST),0.010846,0.674620,0.010846,1.0,1.482315,0.003529,inf,0.328947
103674,"(22352, 22728, 22554)","(22551, 22726)",0.010846,0.032538,0.010846,1.0,30.733333,0.010493,inf,0.978070
103729,"(22352, 22726, 22727, 22556)",(22551),0.010846,0.119306,0.010846,1.0,8.381818,0.009552,inf,0.890351
103728,"(22352, 22726, 22551, 22556)",(22727),0.010846,0.078091,0.010846,1.0,12.805556,0.009999,inf,0.932018
121397,"(21086, 22492, 22333, POST, 21094)",(21080),0.010846,0.112798,0.010846,1.0,8.865385,0.009623,inf,0.896930
52539,"(22326, 22328, 21731)",(POST),0.010846,0.674620,0.010846,1.0,1.482315,0.003529,inf,0.328947
128848,"(21094, 22726, 22727, 22556)","(22326, 22728)",0.010846,0.041215,0.010846,1.0,24.263158,0.010399,inf,0.969298
103705,"(22352, 22727, 22554)","(22551, 22728)",0.010846,0.032538,0.010846,1.0,30.733333,0.010493,inf,0.978070
138543,"(22551, 22730, 22728)","(22726, 22727, 22556)",0.010846,0.021692,0.010846,1.0,46.100000,0.010611,inf,0.989035
103703,"(22352, 22728, 22554)","(22551, 22727)",0.010846,0.030369,0.010846,1.0,32.928571,0.010517,inf,0.980263


In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import matplotlib.pyplot as plt
# 6. Filter out the rating record for the movies 
movies_df = pd.read_csv('movies.csv')
movies_df.info()

ratings_df = pd.read_csv('ratings.csv')
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [18]:
# Filter ratings for movies in the "Action" genre
action_movies = movies_df[movies_df['genres'].str.contains('Action')]
action_ratings = ratings_df[ratings_df['movieId'].isin(action_movies['movieId'])]

# Group by movie and calculate the average rating
average_ratings = action_ratings.groupby('movieId')['rating'].mean().reset_index()

# Sort the movies by average rating in descending order
sorted_action_movies = average_ratings.sort_values(by='rating', ascending=False)

# Filter out the movies already rated by users
sorted_action_movies = sorted_action_movies[~sorted_action_movies['movieId'].isin(action_ratings['movieId'])]

# Recommend the top 5 movies
top_5_recommended_movies = sorted_action_movies.head(5)

print("Top 5 recommended action movies:")
print(top_5_recommended_movies)

Top 5 recommended action movies:
Empty DataFrame
Columns: [movieId, rating]
Index: []
